### Force all person names to be one of six names

The source data came from 1,161 texts, so the generated sentences contained a welter of character names.  To make the result seem a little more consistent, I run the generated sentences through the Stanford corenlp package (Stanford's named-entity recognition worked much better here than spacy's), map the resulting names to my six character names, etc.

In [33]:
import codecs, re, glob

stories = []
file_names = []

for p in glob.glob('raw_stories/*.txt'):
    file_names.append(p.split('/')[-1])
    stories.append(re.split('\n', codecs.open(p, 'r', encoding='utf-8').read()))
    
print 'len(stories)', len(stories)
print file_names[0]

len(stories) 100
58.txt


In [2]:
import re, commands, codecs
from collections import defaultdict, Counter

def process_story(s):

    f = codecs.open('temp_sentences.txt', 'w', encoding='utf-8')
    f.write('\n'.join(s) + '\n')
    f.close()

    cmd_results = commands.getoutput('cd /home/spenteco/0/stanford-corenlp-full-2018-10-05;' + \
                                './corenlp.sh -annotators tokenize,ssplit,pos,lemma,ner -outputFormat conll -file ' + \
                                 '/home/spenteco/1/nanogenmo_2017/temp_sentences.txt' + \
                                 ' -outputDirectory /home/spenteco/1/nanogenmo_2017')

    f = codecs.open('temp_sentences.txt.conll', 'r', encoding='utf-8')
    result_sentences =  re.split('\n\n', f.read())
    f.close()
    
    persons = []
    locations = []
    organizations = []
    
    for r in result_sentences:
    
        name_type = ''
        name_parts = []
    
        for t in r.split('\n'):

            cols = t.split('\t')
            if len(cols) > 4:

                text = cols[1]
                lemma = cols[2]
                pos = cols[3]
                ent = cols[4]
            
            if ent in ['PERSON', 'LOCATION', 'ORGANIZATION'] and \
                text not in ['\'s', '--', 'bush', 'messenger', 'sun', 'earth', 'moon', 'of']:
                    
                if name_type > '' and name_type != ent:
                
                    if name_type == 'PERSON':
                        persons.append(' '.join(name_parts))
                    elif name_type == 'LOCATION':
                        locations.append(' '.join(name_parts))
                    elif name_type == 'ORGANIZATION':
                        organizations.append(' '.join(name_parts))
                        
                    name_type = ''
                    name_parts = []
                    
                else:
                    name_parts.append(text)
                    name_type = ent
            else:
                if name_type > '':
                
                    if name_type == 'PERSON':
                        persons.append(' '.join(name_parts))
                    elif name_type == 'LOCATION':
                        locations.append(' '.join(name_parts))
                    elif name_type == 'ORGANIZATION':
                        organizations.append(' '.join(name_parts))
                        
                    name_type = ''
                    name_parts = []
    
        
    if name_type > '':

        if name_type == 'PERSON':
            persons.append(' '.join(name_parts))
        elif name_type == 'LOCATION':
            locations.append(' '.join(name_parts))
        elif name_type == 'ORGANIZATION':
            organizations.append(' '.join(name_parts))

        name_type = ''
        name_parts = []
    
    #persons = sorted(list(set(persons)))
    #locations = sorted(list(set(locations)))
    #organizations = sorted(list(set(organizations)))

    return persons, locations, organizations

In [3]:
# http://www.scifiideas.com/sci-fi-character-name-generator/

import random

raw_names = ['Killian Masood', 'Kamal Rainbird', 'Cyrus Rana', 'Ace Kelly', 
             'Rhea Landeros', 'Tekla Tanner', 'Ellia Alpha-2-Guthren']

new_names = []
for n in raw_names:
    new_names.append([n, n.split(' ')[0]])

print new_names

[['Killian Masood', 'Killian'], ['Kamal Rainbird', 'Kamal'], ['Cyrus Rana', 'Cyrus'], ['Ace Kelly', 'Ace'], ['Rhea Landeros', 'Rhea'], ['Tekla Tanner', 'Tekla'], ['Ellia Alpha-2-Guthren', 'Ellia']]


In [4]:
import re, commands, codecs
from collections import defaultdict, Counter
    
all_original_names = []
    
for sn, s in enumerate(stories):
    
    print sn,
    
    persons, locations, organizations = process_story(s)
    
    all_original_names.append(persons)
    
print

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99


In [19]:
print all_original_names[0]

[u'Mel', u'Dominico', u'Arek', u'Mel', u'Roger', u'Bastin', u'Bailey', u'Walters', u'Mel', u'Dominico', u'Nanlo', u'Nanlo', u'Mel', u'Merrywinkle', u'Masterson', u'Madonna']


In [20]:
all_name_mappings = []

for a, names in enumerate(all_original_names):
    
    name_counts = defaultdict(int)
    for n in names:
        name_counts[n] += 1
    
    random.shuffle(new_names)
    
    names_to_use = new_names[:3]
    
    new_i = -1
    
    name_mappings = []
    
    for w in Counter(name_counts).most_common():
        new_i += 1
        if new_i > len(names_to_use) - 1:
            new_i = 0
        name_mappings.append([w[0], names_to_use[new_i]])
        
        
    all_name_mappings.append(name_mappings)
    
print 'len(all_name_mappings)', len(all_name_mappings)

print all_name_mappings[0]

len(all_name_mappings) 100
[[u'Mel', ['Ellia Alpha-2-Guthren', 'Ellia']], [u'Dominico', ['Rhea Landeros', 'Rhea']], [u'Nanlo', ['Ace Kelly', 'Ace']], [u'Bastin', ['Ellia Alpha-2-Guthren', 'Ellia']], [u'Masterson', ['Rhea Landeros', 'Rhea']], [u'Merrywinkle', ['Ace Kelly', 'Ace']], [u'Arek', ['Ellia Alpha-2-Guthren', 'Ellia']], [u'Roger', ['Rhea Landeros', 'Rhea']], [u'Bailey', ['Ace Kelly', 'Ace']], [u'Madonna', ['Ellia Alpha-2-Guthren', 'Ellia']], [u'Walters', ['Rhea Landeros', 'Rhea']]]


In [34]:
import re

for sn, s in enumerate(stories):
    
    named_story = []
    last_names_used = []
    
    for sentence in s:
        
        new_sentence = sentence
    
        for name in all_name_mappings[sn]:

            matches = []

            for mn, m in enumerate(re.finditer(r'\b' + name[0] + r'\b', new_sentence)):

                matches.append([m.start(), m.end()])

            matches.reverse()

            for mn, m in enumerate(matches):

                if name[1][0] in last_names_used:
                    new_sentence = new_sentence[:m[0]] + name[1][1] + new_sentence[m[1]:]
                else:
                    new_sentence = new_sentence[:m[0]] + name[1][0] + new_sentence[m[1]:]
                    last_names_used.append(name[1][0])
                    
        named_story.append(new_sentence)
                
    stories[sn] = '\n'.join(named_story)
    
print 'Done!'

Done!


In [35]:
for sn, s in enumerate(stories):
    
    f = codecs.open('named_stories/' + file_names[sn], 'w', encoding='utf-8')
    f.write(stories[sn])
    f.close()
    
print 'Done!'

Done!
